# 推薦模型 - 矩陣分解

在之前的介紹裡面，相似度關係是利用交易資料所形成的向量空間，而對兩兩用戶(或物品)來求得的。但是實務上使用這個方法會碰到問題：

1. 維度過於龐大與稀疏，例如電子商務網站會有上百萬個用品。而單一用戶往往只有很少量(數個）與物品的交互關係(交易紀錄interaction data)此時會形成[維度災難](https://zh.wikipedia.org/wiki/%E7%BB%B4%E6%95%B0%E7%81%BE%E9%9A%BE)，亦即在維度過大的情況下，所有的東西(人)相似度趨近於0(距離無窮遠)
2. 計算用戶（或物品)的兩兩關係會隨著用戶(物品）增加而耗時成指數增加

矩陣分解目的是解決**維度災難**的手法
## 矩陣分解

在矩陣分解的想法裡面，把用戶與物品的交易矩陣，用一種線性關係來逼近，

$$r_{ui} = \textbf{x}_u^T \cdot \textbf{y}_i $$

代表的意思為，用戶購買某商品的背後，有一種隱性特徵來決定購買的權重。而每個商品有關於這個隱性特徵的比例。這樣的線性關係，恰恰決定了用戶對某商品的打分。

- 舉例來說: 小明會給**超人特攻隊** `評分=5`, 原因可能是這部片背後有三種特徵: $\textbf{y}_{超人特攻隊}=$ `{ 恐怖：0, 喜劇：2, 卡通:3 }`,而小明對這三種特徵的喜好程度分別是: $\textbf{x}_{小明}$ = `{喜愛恐怖:0, 喜愛喜劇:0.9, 喜愛卡通: 1}`。按照矩陣分解的想法：

$$r_{小明-超人特攻隊} = \textbf{x}_{小明}^T \cdot \textbf{y}_{超人特攻隊} = 1.8 + 3 = 4.8 \approx 5$$

- 損失函數可寫成 
$$
L = \sum_{u,i \in S} \left(r_{ui} - \textbf{x}_u \cdot \textbf{y}_i\right)^2 + \lambda_x \sum_u \left\Vert \textbf{x}_u \right\Vert ^2  + \lambda_y \sum_i \left\Vert \textbf{y}_i \right\Vert ^2 $$

> 集合$s$表示有評分的物件(交互作用),$x_u,y_i$分別表示用戶$u$(物品$i$)的向量表示,$\lambda_x,\lambda_y$表示regularization

### explicit ALS 算法

- 要最小化此損失函數，可以先固定 $\textbf{y}_i$為常數對另一變數$\textbf{x}_u$進行微分，並另其為0求得關係...
- 相似的固定 $\textbf{x}_u$為常數，對另一變數$\textbf{y}_i$進行微分。
- 重複上述動作直到收斂

上述過程稱為Alternative Least Square (ALS)算法，由於物標函數是評價分數(1-5)的明顯用戶回饋分數，所以稱為explicit ALS算法。概念上的框架是基於矩陣分解，而針對兩組方程用交互迭代的方式取得收斂。

## Implicit ALS 算法

其實很常見的狀況是無從知道到底用戶對商品的評價是什麼，只能隱約猜測有買過的東西，對其偏好(preference)程度較高。但是對於沒有買過的商品，可能是因為
1. 不喜歡此類商品
2. 未察覺此商品

在此篇[論文](http://yifanhu.net/PUB/cf.pdf)中提出信心程度的想法，將explicit ALS做進一步的改良。

$$
L = \sum_{u,i \in all} c_{ui}\left(p_{ui} - \textbf{x}_u \cdot \textbf{y}_i\right)^2 + \lambda_x \sum_u \left\Vert \textbf{x}_u \right\Vert ^2  + \lambda_y \sum_i \left\Vert \textbf{y}_i \right\Vert ^2 
$$

其中$p_{ui}$為喜歡或不喜歡${0,1}$之偏好，而$c_{ui}$代表說明用戶$u$對商品$i$之說明喜歡(或不喜歡)的信心程度，數值愈高代表信心程度愈大。與之前僅考慮有交互作用($\in S$)的情況不同，需要考慮所有未購買的狀況($\in all_{ui}$。類似explicit解法，可以求得解析解

$$
\textbf{X}_u = \left( \textbf{Y}^T\textbf{C}^u\textbf{Y}  + \lambda \textbf{I}\right)^{-1} \textbf{Y}^T\textbf{C}^uP(u)
$$
$$
\textbf{Y}_i = \left( \textbf{X}^T\textbf{C}^i\textbf{X}  + \lambda \textbf{I}\right)^{-1} \textbf{X}^T\textbf{C}^iP(i)
$$

In [2]:
import numpy as np 
import pandas as pd
import csv
import sys
from tqdm import tqdm
sys.path.append('../')

In [3]:
from rec_helper import *

In [4]:
df = pd.read_csv('../rec-a-sketch/model_likes_anon.psv',
                 sep='|',quotechar='\\',quoting=csv.QUOTE_MINIMAL)
print(df.count())
df.drop_duplicates(inplace=True)
print(df.count())
df = threshold_interaction(df,rowname='uid',colname='mid',row_min=5,col_min=10)
inter,uid_to_idx,idx_to_uid,mid_to_idx,idx_to_mid=df_to_spmatrix(df,'uid','mid')
train,test, user_idxs = train_test_split(inter,split_count=1,fraction=0.2)

modelname    632832
mid          632832
uid          632832
dtype: int64
modelname    632677
mid          632677
uid          632677
dtype: int64
Starting interactions info
Number of rows: 62583
Number of cols: 28806
Sparsity: 0.04%
Ending interactions info
Number of rows: 13496
Number of columns: 13618
Sparsity: 0.25%


In [5]:
def alternating_least_squares(Cui, factors, regularization, iterations=20):
    users, items = Cui.shape

    X = np.random.rand(users, factors) * 0.01
    Y = np.random.rand(items, factors) * 0.01

    Ciu = Cui.T.tocsr()
    for iteration in range(iterations):
        X,Y = least_squares(Cui, X, Y, regularization)
        Y,X = least_squares(Ciu, Y, X, regularization)
        print('iter:{}'.format(iteration))

    return X, Y

In [6]:
def least_squares(Cui, X, Y, regularization):
    users, factors = X.shape
    YtY = Y.T.dot(Y)

    for u in range(users):
        # accumulate YtCuY + regularization * I in A
        A = YtY + regularization * np.eye(factors)

        # accumulate YtCuPu in b
        b = np.zeros(factors)
#         if u % 1000 == 0:
#             print(u)
        for i in Cui[u,:].indices:
            confidence = Cui[u,i]
            factor = Y[i]
            A += (confidence - 1) * np.outer(factor, factor)
            b += confidence * factor

        # Xu = (YtCuY + regularization * I)^-1 (YtCuPu)
        X[u] = np.linalg.solve(A, b)
    return X,Y

In [7]:
users_embedding, items_embedding = alternating_least_squares(train,50,regularization=1,iterations=10)

iter:0
iter:1
iter:2
iter:3
iter:4
iter:5
iter:6
iter:7
iter:8
iter:9


In [8]:
import annoy

In [14]:
class ApproximateTopRelated:
    def __init__(self, items_factors, treecount=20):
        index = annoy.AnnoyIndex(items_factors.shape[1], 'angular')
        for i, row in enumerate(items_factors):
            index.add_item(i, row)
        index.build(treecount)
        self.index = index

    def get_related(self, itemid, N=10):
        neighbours = self.index.get_nns_by_item(itemid, N)
        return sorted(((other, 1 - self.index.get_distance(itemid, other))
                      for other in neighbours), key=lambda x: -x[1])

In [16]:
approx_topRelated_item = ApproximateTopRelated(items_embedding)

In [17]:
approx_topRelated_item.get_related(10)

[(10, 1.0),
 (73, 0.4507659077644348),
 (7, 0.2394477128982544),
 (56, 0.2023945450782776),
 (51, 0.17766046524047852),
 (13005, 0.14669829607009888),
 (11590, 0.10458815097808838),
 (19, 0.08599740266799927),
 (107, 0.05054211616516113),
 (11270, 0.04526710510253906)]

In [22]:
import requests
def get_thumbnails(approx_top_related_items, idx, idx_to_mid, N=10):
#     row = sim[idx, :].A.ravel()
    topNitems,scores = zip(*approx_top_related_items.get_related(idx))
    thumbs = []
    for x in topNitems:
        response = requests.get('https://sketchfab.com/i/models/{}'.format(idx_to_mid[x])).json()
        thumb = [x['url'] for x in response['thumbnails']['images'] if x['width'] == 200 and x['height']==200]
        if not thumb:
            print('no thumbnail')
        else:
            thumb = thumb[0]
        thumbs.append(thumb)
    return thumbs

In [23]:
thumbs = get_thumbnails(approx_topRelated_item, idx=0, idx_to_mid=idx_to_mid)

no thumbnail
no thumbnail
no thumbnail


In [25]:
from IPython.display import HTML, display

In [26]:
def display_item(thumbs,N=5):
    try: 
        thumb_html = '<img src='+ '\"'+thumbs[0]+'\">' 
    except TypeError:
        print('No thumbnail...origin')
        thumb_html= ""
    for url in thumbs[1:]:
        if url:
            thumb_html = thumb_html + '<img src='+ '\"'+  url + '\">'     
    return thumb_html

In [37]:
HTML(display_item(thumbs))

No thumbnail...origin
